# Assignment 10

1. Build a dataset using the files provided on the lecture page (bmi.skku.edu). The provided dataset consists of biosignals collected at 1-minute intervals and their labels divided into 60/40 people, and no additional pre-processing is required.
2. Create any RNN series model and train it with the train dataset. The type and performance of the model do not affect the score.

3. Evaluate the trained model with test dataset, and in particular, you have to take one sample of data from the test set, feed it into the model, and show the label and prediction.

## 1 Build Dataset

In [1]:
from google.colab import drive
from PIL import Image
import os
import numpy as np


drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
import pandas as pd

test_data_path = '/content/gdrive/MyDrive/Assignment#10_DHC/homework_test_data.csv'
test_label_path = '/content/gdrive/MyDrive/Assignment#10_DHC/homework_test_label.csv'
train_data_path = '/content/gdrive/MyDrive/Assignment#10_DHC/homework_train_data.csv'
train_label_path = '/content/gdrive/MyDrive/Assignment#10_DHC/homework_train_label.csv'


x_train = np.array(pd.read_csv(train_data_path, header=None))
y_train = np.array(pd.read_csv(train_label_path, header=None))

x_test = np.array(pd.read_csv(test_data_path, header=None))
y_test = np.array(pd.read_csv(test_label_path, header=None))

x_train = x_train.reshape((x_train.shape[0],1,x_train.shape[1]))

x_test = x_test.reshape((x_test.shape[0],1,x_test.shape[1]))

In [17]:

print(f'Train Shape: {x_train.shape} | Test Shape: {x_test.shape}')

Train Shape: (60, 1, 600) | Test Shape: (40, 1, 600)


## 2 RNN Model

In [37]:
import tensorflow as tf
from tensorflow import keras

class Recurrent(keras.Model):
  def __init__(self):
    super(Recurrent, self).__init__()

    self.lstm = keras.layers.LSTM(600, return_sequences=True, return_state=False)
    
    self.head = keras.layers.Dense(1, activation='sigmoid')
  
  def call(self, x):
    x = self.lstm(x)

    x = keras.layers.Flatten()(x)
    x = self.head(x)
    return x



In [40]:
optim = keras.optimizers.Adam(learning_rate=0.0002)
loss = keras.losses.BinaryCrossentropy()

model = Recurrent()
model.compile(
    optimizer=optim,
    loss=loss,
    metrics=['accuracy'],
)

In [44]:
model.fit(x=x_train, y=y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
2/2 [==============================] - 0s 90ms/step - loss: 0.6067 - accuracy: 0.7500 - val_loss: 0.6742 - val_accuracy: 0.6750
Epoch 2/10
2/2 [==============================] - 0s 64ms/step - loss: 0.6056 - accuracy: 0.8000 - val_loss: 0.6846 - val_accuracy: 0.6250
Epoch 3/10
2/2 [==============================] - 0s 56ms/step - loss: 0.6051 - accuracy: 0.8000 - val_loss: 0.6838 - val_accuracy: 0.6500
Epoch 4/10
2/2 [==============================] - 0s 64ms/step - loss: 0.6038 - accuracy: 0.8000 - val_loss: 0.6877 - val_accuracy: 0.5750
Epoch 5/10
2/2 [==============================] - 0s 60ms/step - loss: 0.5969 - accuracy: 0.8333 - val_loss: 0.6964 - val_accuracy: 0.4500
Epoch 6/10
2/2 [==============================] - 0s 57ms/step - loss: 0.6032 - accuracy: 0.7833 - val_loss: 0.7005 - val_accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 0s 60ms/step - loss: 0.5930 - accuracy: 0.8000 - val_loss: 0.6925 - val_accuracy: 0.5250
Epoch 8/10
2/2 [===========

## 3 Model Evaluation

In [45]:
model.evaluate(x=x_test, y=y_test)

2/2 [==============================] - 0s 10ms/step - loss: 0.6726 - accuracy: 0.6750


[0.6726350784301758, 0.675000011920929]

In [53]:
test_snippet = x_test[0].reshape(1,x_test.shape[1], x_test.shape[2])
predict = model.predict(test_snippet)

print(f'Prediction Probability | Ground Truth : {predict} | {y_test[0]}')
print(f'Hard Prediction | Ground Truth : {np.argmax(predict)} | {y_test[0]}')

Prediction Probability | Ground Truth : [[0.46460888]] | [0]
Hard Prediction | Ground Truth : 0 | [0]
